In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot= True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def weights(shape):
    random_weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(random_weights)
    

In [5]:
def bias(shape):
    bias_const = tf.constant(0,1, shape= shape)
    return tf.Variable(bias_const)

In [6]:
def filters(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [7]:
def maxpool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
    

In [8]:
def convol_layer(input_img, shape):
    W = weights(shape)
    b = bias([shape[3]])
    return tf.nn.relu(filters(input_img, W) + b)

In [9]:
def normal_layer(input_x, size):
    input_size = int(input_x.get_shape()[1])
    W = weights([input_size, size])
    b = bias([size])
    return tf.matmul(input_x, W) + b
    

In [10]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None , 10])

In [11]:
x_img = tf.reshape(X, shape=[-1,28, 28, 1])

In [12]:
# FIRST CONVOLUTIONAL LAYER

In [13]:
conv_1 = convol_layer(x_img, [6,6,1,32])

In [14]:
conv_1_pool = maxpool(conv_1)

In [15]:
# SECOND CONVOLUTIONAL LAYER

In [16]:
conv_2 = convol_layer(conv_1_pool, [6,6,32,64])

In [17]:
conv_2_pool = maxpool(conv_2)

In [18]:
conv_flat = tf.reshape(conv_2_pool,[-1,7*7*64])

In [19]:
full_one_layer = tf.nn.relu(normal_layer(conv_flat, 1024))

In [20]:
hold_prob = tf.placeholder(tf.float32)

In [21]:
full_dropout = tf.nn.dropout(full_one_layer, keep_prob=hold_prob)

In [22]:
y_pred =  normal_layer(full_dropout, 10)

In [23]:
# Loss function

In [24]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits=y_pred ))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [26]:
init = tf.global_variables_initializer()

In [ ]:
steps = 1
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x , batch_y =mnist.train.next_batch(1)
        
        print("Training images step ()".format(i))
        sess.run(train, feed_dict={X:batch_x, y:batch_y, hold_prob:0.5})

        matches = tf.equal(tf.argmax(y,1), tf.argmax(y_pred,1))
        acc = tf.reduce_mean(tf.cast(matches, tf.float32))

        print(sess.run(acc, feed_dict={X:mnist.test.images, y:mnist.test.labels, hold_prob:0.5}))
   
        

Training images step ()
